In [22]:
!pip install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

def init_spark(app_name: str):
  spark = SparkSession.builder.appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

spark, sc = init_spark('demo')


In [25]:
data_file = 'Lab3_view_data.csv'
Lab3_view_data_rdd = sc.textFile(data_file)
print('Type:', type(Lab3_view_data_rdd))
print('Count:', Lab3_view_data_rdd.count())

Type: <class 'pyspark.rdd.RDD'>
Count: 1048576


Clean the column names:

In [26]:
csv_rdd=Lab3_view_data_rdd.map(lambda row: row.split(','))
header=csv_rdd.first()
print(header)
clean_data = csv_rdd.filter(lambda row: row != header)


['mso_code ', 'device_id ', 'event_date ', 'event_time', 'station_num', 'prog_code']


Solution:

In [30]:
prime_time_data = clean_data.filter(lambda row: int(row[3]) >=200000 and (int(row[3]) <=225959))

# Map each device-date-time trio to a count of the number of events
device_date_events_rdd = prime_time_data.map(lambda line: ((line[1], line[2],line[3]), 1))
device_date_count_rdd = device_date_events_rdd.reduceByKey(lambda a, b: a + b)
# Map each device to the total number of events across all dates
device_total_events_rdd = device_date_count_rdd.map(lambda x: (x[0][0], x[1]))

device_total_events_rdd = device_total_events_rdd.reduceByKey(lambda a, b: a + b)
# device_total_events_rdd.collect()

# # Count the number of unique dates
unique_dates_count = prime_time_data.map(lambda line: line[2]).distinct().count()

# # # Map each device to the average number of events per date
device_avg_events_rdd = device_total_events_rdd.map(lambda x: (x[0], x[1] / unique_dates_count))
# device_avg_events_rdd.collect()

# # device_avg_events_rdd.collect()
top_5_devices = device_avg_events_rdd.sortBy(lambda x: x[1], ascending=False).take(5)

# # # Get the top 5 devices with the highest average number of events
for device in top_5_devices:
  print(device)

('7.5E+14', 97.64285714285714)
('7.46E+14', 11.714285714285714)
('7.503E+14', 9.357142857142858)
('8.00001E+11', 7.5)
('8.4843E+14', 5.5)
